#LSTM 一維時序信號訓練及預測範例  
歐尼克斯實境互動工作室 OmniXRI Jack, 2021.6.15  
參考資料：Github lucko515/tesla-stocks-prediction  
https://github.com/lucko515/tesla-stocks-prediction

強迫Colab使用TensorFlow 1.x版以符合後續程式運行    
宣告必要使用函式庫  

In [ ]:
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline

print(tf.__version__) # 確認TensorFlow版本

#1.下載資料集並進行預處理
將資料集讀入並顯示開頭數筆完整資料  
只抽取'Close'一欄資料作為後續範例使用並顯示資料集筆數  

In [ ]:
# 下載測試資料集  
# 使用 !wget https://raw.githubusercontent.com/lucko515/tesla-stocks-prediction/master/tesla_stocks.csv
# 取代 !wget https://github.com/lucko515/tesla-stocks-prediction/blob/master/tesla_stocks.csv

!wget -N https://raw.githubusercontent.com/lucko515/tesla-stocks-prediction/master/tesla_stocks.csv
!ls

# 列出資料集完整內容
#!cat tesla_stocks.csv

# 讀取完整資料集
tesla_stocks = pd.read_csv('tesla_stocks.csv')
# 顯示頭幾筆完整資料
tesla_stocks.head()

In [ ]:
# 只抽取'Close'欄位資料作為後續使用
data_to_use = tesla_stocks['Close'].values
# 列出資料集筆數
print('Total number of days in the dataset: {}'.format(len(data_to_use)))

#2.將資料正規化並繪製圖示

In [ ]:
# 將資料正規化
scaler = StandardScaler()
scaled_dataset = scaler.fit_transform(data_to_use.reshape(-1, 1))
# 繪製原始資料
plt.figure(figsize=(12,7), frameon=False, facecolor='brown', edgecolor='blue')
plt.title('Scaled TESLA stocks from August 2014 to August 2017')
plt.xlabel('Days')
plt.ylabel('Scaled value of stocks')
plt.plot(scaled_dataset, label='Stocks data')
plt.legend()
plt.show()

#window_data()函式
以移動視窗方式提取輸入及輸出資料，以利後續訓練及驗證用。  

In [ ]:
def window_data(data, window_size):
    X = [] # 取得輸入資料，window_size筆資料為一組
    y = [] # 取得輸出資料，window_size後的一筆
    
    i = 0
    while (i + window_size) <= len(data) - 1:
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
        
        i += 1
    assert len(X) == len(y)
    return X, y

宣告程式所需主要參數

In [ ]:
epochs = 200 # 訓練次數
batch_size = 7 # 資料批次提取數量
train_amount = 700 # 訓練資料數量

#3.取得訓練和測試資料集

In [ ]:
# 取得訓練及測試資料
X, y = window_data(scaled_dataset, batch_size)
# 取得train_amount筆數資料作為訓練集
X_train  = np.array(X[:train_amount])
y_train = np.array(y[:train_amount])
# 剩下的部份作為測試集
X_test = np.array(X[train_amount:])
y_test = np.array(y[train_amount:])
# 列出資料集數量
print("X_train size: {}".format(X_train.shape))
print("y_train size: {}".format(y_train.shape))
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

#LSTM_cell()函式  


In [ ]:
def LSTM_cell(hidden_layer_size, batch_size,number_of_layers, dropout=True, dropout_rate=0.8):
    
    layer = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size)
    
    if dropout:
        layer = tf.contrib.rnn.DropoutWrapper(layer, output_keep_prob=dropout_rate)
        
    cell = tf.contrib.rnn.MultiRNNCell([layer]*number_of_layers)
    
    init_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, init_state

#output_layer()函式  

In [ ]:
def output_layer(lstm_output, in_size, out_size):
    
    x = lstm_output[:, -1, :]
    print(x)
    weights = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.05), name='output_layer_weights')
    bias = tf.Variable(tf.zeros([out_size]), name='output_layer_bias')
    
    output = tf.matmul(x, weights) + bias
    return output

#opt_loss()函式

In [ ]:
def opt_loss(logits, targets, learning_rate, grad_clip_margin):
    
    losses = []
    for i in range(targets.get_shape()[0]):
        losses.append([(tf.pow(logits[i] - targets[i], 2))])
        
    loss = tf.reduce_sum(losses)/(2*batch_size)
    
    #Cliping the gradient loss
    gradients = tf.gradients(loss, tf.trainable_variables())
    clipper_, _ = tf.clip_by_global_norm(gradients, grad_clip_margin)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_optimizer = optimizer.apply_gradients(zip(gradients, tf.trainable_variables()))
    return loss, train_optimizer

#StockPredictionRNN()類別函式


In [ ]:
class StockPredictionRNN(object):
    
    def __init__(self, learning_rate=0.001, batch_size=7, hidden_layer_size=512, number_of_layers=1, 
                 dropout=True, dropout_rate=0.8, number_of_classes=1, gradient_clip_margin=4, window_size=7):
    
        self.inputs = tf.placeholder(tf.float32, [batch_size, window_size, 1], name='input_data')
        self.targets = tf.placeholder(tf.float32, [batch_size, 1], name='targets')

        cell, init_state = LSTM_cell(hidden_layer_size, batch_size, number_of_layers, dropout, dropout_rate)

        outputs, states = tf.nn.dynamic_rnn(cell, self.inputs, initial_state=init_state)

        self.logits = output_layer(outputs, hidden_layer_size, number_of_classes)

        self.loss, self.opt = opt_loss(self.logits, self.targets, learning_rate, gradient_clip_margin)

設定model, session

In [ ]:
tf.reset_default_graph()
model = StockPredictionRNN()

session =  tf.Session()
session.run(tf.global_variables_initializer())

#開始訓練

In [ ]:
import time

time_start = time.time() #開始計時

for i in range(epochs):
    traind_scores = []
    ii = 0
    epoch_loss = []
    while(ii + batch_size) <= len(X_train):
        X_batch = X_train[ii:ii+batch_size]
        y_batch = y_train[ii:ii+batch_size]
        
        o, c, _ = session.run([model.logits, model.loss, model.opt], feed_dict={model.inputs:X_batch, model.targets:y_batch})
        
        epoch_loss.append(c)
        traind_scores.append(o)
        ii += batch_size
    if (i % 10) == 0: # 每隔10次列出目前損失率，次數可自行調整
        print('Epoch {}/{}'.format(i, epochs), ' Current loss: {}'.format(np.mean(epoch_loss)))

time_end = time.time() # 結束計時
time_c= time_end - time_start # 計算執行所花時間
print('time cost', time_c, 's') # 列出總共花費時間

取得正規化後原始資料

In [ ]:
sup =[]
for i in range(len(traind_scores)):
    for j in range(len(traind_scores[i])):
        sup.append(traind_scores[i][j])

取得測試資料

In [ ]:
tests = []
i = 0
while i+batch_size <= len(X_test):
    
    o = session.run([model.logits], feed_dict={model.inputs:X_test[i:i+batch_size]})
    i += batch_size
    tests.append(o)

產生最後測試資料

In [ ]:
tests_new = []
for i in range(len(tests)):
    for j in range(len(tests[i][0])):
        tests_new.append(tests[i][0][j])

分別產生訓練和測試結果資料序列

In [ ]:
test_results = []
for i in range(749):
    if i >= 701:
        test_results.append(tests_new[i-701])
    else:
        test_results.append(None)

#繪製出結果
藍色線為原始資料正規化內容，橙色線訓練集資料，綠色為測試輸出結果。

In [ ]:
plt.figure(figsize=(16, 7))
plt.plot(scaled_dataset, label='Original data')
plt.plot(sup, label='Training data')
plt.plot(test_results, label='Testing data')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 7))
plt.plot(epoch_loss, label='Epoch Loss')
plt.legend()
plt.show()

In [ ]:
session.close()